## Data Input and Cleaning 

In [1]:
import pandas as pd
import numpy as np

# Read CSV
planet_df = pd.read_csv('exoplanet_data.csv')
planet_df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [2]:
# Drop the null columns where all values are null
clean_df = planet_df.dropna(axis='columns', how='all')

# Drop the null rows
clean_df = clean_df.dropna()
clean_df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Feature Selection and Pre-processing 

In [3]:
# Pull in important features from forest model
%store -r important_features
%store -r feature_importance_list

In [4]:
important_features.remove('koi_disposition')
print(important_features)

['koi_fpflag_ss', 'koi_fpflag_nt', 'koi_insol_err2', 'koi_depth_err2', 'koi_duration', 'koi_fpflag_co', 'koi_prad', 'koi_duration_err1', 'koi_prad_err1', 'koi_steff', 'koi_steff_err1', 'koi_time0bk', 'koi_impact_err2', 'koi_time0bk_err1', 'koi_fpflag_ec', 'koi_time0bk_err2', 'koi_period', 'koi_duration_err2', 'koi_period_err1', 'koi_insol', 'koi_prad_err2', 'koi_teq', 'koi_depth', 'koi_depth_err1', 'koi_period_err2', 'koi_srad_err2', 'ra', 'koi_insol_err1', 'koi_impact', 'koi_srad', 'koi_slogg_err1', 'koi_impact_err1', 'dec', 'koi_tce_plnt_num', 'koi_slogg_err2', 'koi_slogg', 'koi_steff_err2', 'koi_srad_err1', 'koi_model_snr']


In [5]:
# List labels
label_names = []
for n in  clean_df['koi_disposition'].unique():
    label_names.append(n)

In [6]:
# Define X and y
labels = clean_df['koi_disposition']
features = clean_df.drop('koi_disposition', axis=1)

# Pick forest's most important features
top_ten_features = features[important_features[:10]]
features = top_ten_features

X = features.values
y = labels.values

In [7]:
print(X.shape, y.shape)

(6991, 10) (6991,)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [9]:
 X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [10]:
# Scale
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [12]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Building the Deep Learning Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
 # Create model and add layers
DL_model = Sequential()
DL_model.add(Dense(units=100, activation='relu', input_dim=10))
DL_model.add(Dense(units=100, activation='relu'))
DL_model.add(Dense(units=100, activation='relu'))
DL_model.add(Dense(units=3, activation='softmax'))

In [15]:
 # Compile and fit the model
DL_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
print(X_train_scaled.shape, y_train_categorical.shape)

(5243, 10) (5243, 3)


## Training the Model

In [17]:
DL_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/60
5243/5243 - 2s - loss: 0.5432 - accuracy: 0.6954
Epoch 2/60
5243/5243 - 0s - loss: 0.4494 - accuracy: 0.7446
Epoch 3/60
5243/5243 - 0s - loss: 0.4452 - accuracy: 0.7532
Epoch 4/60
5243/5243 - 0s - loss: 0.4358 - accuracy: 0.7719
Epoch 5/60
5243/5243 - 1s - loss: 0.4277 - accuracy: 0.7841
Epoch 6/60
5243/5243 - 0s - loss: 0.4136 - accuracy: 0.8098
Epoch 7/60
5243/5243 - 0s - loss: 0.3951 - accuracy: 0.8240
Epoch 8/60
5243/5243 - 0s - loss: 0.3901 - accuracy: 0.8323
Epoch 9/60
5243/5243 - 0s - loss: 0.3844 - accuracy: 0.8325
Epoch 10/60
5243/5243 - 0s - loss: 0.3725 - accuracy: 0.8436
Epoch 11/60
5243/5243 - 0s - loss: 0.3609 - accuracy: 0.8570
Epoch 12/60
5243/5243 - 0s - loss: 0.3638 - accuracy: 0.8478
Epoch 13/60
5243/5243 - 0s - loss: 0.3549 - accuracy: 0.8602
Epoch 14/60
5243/5243 - 0s - loss: 0.3620 - accuracy: 0.8514
Epoch 15/60
5243/5243 - 0s - loss: 0.3549 - accuracy: 0.8596
Epoch 16/60
5243/5243 - 0s - loss: 0.3527 - accuracy: 0.8592
Epoch 17/60

## Evaluating the Model 

In [18]:
model_loss, model_accuracy = DL_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.3339 - accuracy: 0.8707
Normal Neural Network - Loss: 0.3338998756637835, Accuracy: 0.8707093596458435


In [19]:
encoded_predictions = DL_model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [20]:
 print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CANDIDATE'
 'FALSE POSITIVE']
Actual Labels: ['CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE']
